In [30]:
import pandas as pd
import numpy as np
from collections import Counter
import random
from sklearn.feature_extraction import DictVectorizer as DV
from xgboost import XGBClassifier

from bayes_opt.observer import JSONLogger
from bayes_opt.event import Events
from tqdm import tqdm
from matplotlib import pyplot as plt

from sklearn.model_selection import KFold
from catboost.utils import create_cd
from bayes_opt import BayesianOptimization
import copy

In [17]:
from sklearn.metrics import accuracy_score
true = pd.read_csv("true_ans.csv")

def get_ac(ans):
    return accuracy_score(ans, true["Survived"])

In [18]:
data_train = pd.read_pickle("../preprocessing/filled_train.pk")
data_test = pd.read_pickle("../preprocessing/filled_test.pk")

target = data_train["Survived"]

In [19]:
data_train.drop(columns = ["Survived", "PassengerId"], inplace= True)
real_columns = ['Age', 'Fare', "number_of_cabin", "ticket"]
cat_columns = ["Embarked","SibSp", "Parch", 'Pclass','Sex', "Deck", "first_name", "second_name", "title"]
a = list(dict(Counter(data_test["title"])).keys())
idx = data_train.indexidx= []
for row in data_train.iterrows():

    if row[1]["title"] in a:
        idx.append(row[0])
random.shuffle(idx)

data_train["Embarked"] = data_train["Embarked"].fillna("S")
data_test["Embarked"] = data_test["Embarked"].fillna("S")

X_train, X_test, y_train, y_test = data_train.iloc[idx][cat_columns + real_columns], data_test[cat_columns + real_columns], target[idx], true["Survived"]

encoder_1 = DV(sparse = False)
data_cat_oh = encoder_1.fit_transform(X_train[cat_columns].T.to_dict().values())

X_train_n = np.concatenate((X_train[real_columns].values, data_cat_oh), axis = 1)


data_cat_oh = encoder_1.transform(X_test[cat_columns].T.to_dict().values())
X_test_n = np.concatenate((X_test[real_columns].values, data_cat_oh), axis = 1)

/Users/maxim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


In [71]:
def get_model(**params):
    print(params)
    return XGBClassifier(**params)

def validate(model, X_train, y_train, X_val, y_val, verbose=0,n =3):
    model1 = copy.deepcopy(model)
    kfold = KFold(n, True, 1)
    current_score = 0
    for train, test in kfold.split(X_train): 
        model1.fit(X_train_n[train], y_train.values[train], 


        )
        print(model1.get_params())
        probs = model1.predict(X_train_n[test])
        current_score += accuracy_score(probs, y_train.values[test])
    if True:
        print(current_score/n)
        
    return current_score/n

def evaluate_model(**params):
    params_init = {
           "max_depth" : 3,
    "n_estimators": 100,

    "min_child_weight" : 1, 
    }
    params_init.update(params)
    params_init["seed"] = 0
    params_init["n_estimators"] = int(params_init["n_estimators"])
    params_init["max_depth" ] = int(params_init["max_depth" ])
    params_init["min_child_weight" ] = int(params_init["min_child_weight" ])
    model = get_model(**params_init)
    current_score = validate(model, X_train, y_train, X_test, y_test, verbose=0)
    return current_score

In [72]:

pbounds = {
    "max_depth" : (1, 5),
    "n_estimators": (50, 300),

    "min_child_weight" : (1, 5), 
    
    
}

In [73]:
optimizer = BayesianOptimization(evaluate_model, pbounds, random_state=4)

logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTMIZATION_STEP, logger)

optimizer.maximize(init_points=1, n_iter=20)

optimizer.res

# optimized params y

{'max_depth': 4, 'n_estimators': 293, 'min_child_weight': 3, 'seed': 0}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 4, 'min_child_weight': 3, 'missing': None, 'n_estimators': 293, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'silent': None, 'subsample': 1, 'verbosity': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 4, 'min_child_weight': 3, 'missing': None, 'n_estimators': 293, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'silent': None, 'subsample': 1, 'verbosity': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsampl

{'max_depth': 4, 'n_estimators': 83, 'min_child_weight': 1, 'seed': 0}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 4, 'min_child_weight': 1, 'missing': None, 'n_estimators': 83, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'silent': None, 'subsample': 1, 'verbosity': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 4, 'min_child_weight': 1, 'missing': None, 'n_estimators': 83, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'silent': None, 'subsample': 1, 'verbosity': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_b

{'max_depth': 4, 'n_estimators': 189, 'min_child_weight': 4, 'seed': 0}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 4, 'min_child_weight': 4, 'missing': None, 'n_estimators': 189, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'silent': None, 'subsample': 1, 'verbosity': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 4, 'min_child_weight': 4, 'missing': None, 'n_estimators': 189, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'silent': None, 'subsample': 1, 'verbosity': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsampl

{'max_depth': 4, 'n_estimators': 182, 'min_child_weight': 4, 'seed': 0}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 4, 'min_child_weight': 4, 'missing': None, 'n_estimators': 182, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'silent': None, 'subsample': 1, 'verbosity': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 4, 'min_child_weight': 4, 'missing': None, 'n_estimators': 182, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'silent': None, 'subsample': 1, 'verbosity': 1}
{'base_score': 0.5, 'booster': 'gbtree', 'colsampl

[{'target': 0.8283996192333589,
  'params': {'max_depth': 4.868119356054707,
   'min_child_weight': 3.1889289967028893,
   'n_estimators': 293.1710899912211}},
 {'target': 0.8227074675148786,
  'params': {'max_depth': 3.3046238874373524,
   'min_child_weight': 1.9169649152500563,
   'n_estimators': 203.8530647842737}},
 {'target': 0.8113579902950941,
  'params': {'max_depth': 1.210855421990059,
   'min_child_weight': 1.0499620016038,
   'n_estimators': 299.95398587929856}},
 {'target': 0.8227306849929961,
  'params': {'max_depth': 5.0, 'min_child_weight': 5.0, 'n_estimators': 50.0}},
 {'target': 0.8227035979351923,
  'params': {'max_depth': 4.964902947141766,
   'min_child_weight': 4.978175530747913,
   'n_estimators': 122.6159444340648}},
 {'target': 0.8227229458336236,
  'params': {'max_depth': 4.918163180426494,
   'min_child_weight': 4.972734272272998,
   'n_estimators': 249.92772849266026}},
 {'target': 0.8306517146107589,
  'params': {'max_depth': 4.992636911060972,
   'min_child

In [84]:
params= {'max_depth': 4,
   'min_child_weight': 4,
   'n_estimators': 180,
        "seed": 0}
model = XGBClassifier(**params)

In [85]:
model.fit(X_train_n, y_train.values, verbose = True)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=4, missing=None, n_estimators=180, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              silent=None, subsample=1, verbosity=1)

In [86]:
ans = model.predict(X_test_n)

In [87]:
from sklearn.metrics import accuracy_score

In [88]:
accuracy_score(ans, y_test)

0.7655502392344498